## Hands-On Tabular Workshop: Wine classifier with MLFlow, MinIO, and Seldon

In this workshop, we will run an ML Flow model, save the artefacts to MinIO, and create a v2 protocol SeldonDeployment via the SDK

We will follow these steps:
1. Create environment, install libraries, import methods
2. Run experiment and train model in MLFlow
3. Conda pack the environment
4. Push artefacts to MinIO
5. Deploy via the SDK
6. [Optional] Add model metadata via the SDK
7. [Optional] Create a new MLFlow run with different hyperparametes, and deploy a canary model
8. [Optional] Train and deploy a drift detector
9. [Optional] Train and deploy an explainer

### 1. Create environment, install libraries, import methods

In your terminal, create a new virtual environment:
`conda create -n mlflow-wine python=3.8`

Activate the new environment:
`conda activate mlflow-wine`

Check that you are in the correct environment

In [1]:
!conda env list

# conda environments:
#
base                     /home/andrew/.asdf/installs/python/miniconda3-4.7.12
mlflow                   /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/mlflow
mlflow-wine           *  /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/mlflow-wine
python3.8-mlflow-example     /home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/python3.8-mlflow-example
                         /home/andrew/seldon/core-tutorials/iris-explainer-poetry/venv



Install required dependencies into your environment

In [24]:
!pip install seldon_deploy_sdk
!pip install minio
!pip install conda-pack
!pip install mlserver
!pip install mlserver-mlflow
!pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=b90e19987174207651eeb3a14ca250f3761023c927b969a7f1e132d061059834
  Stored in directory: /home/andrew/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [3]:
from seldon_deploy_sdk import Configuration, PredictApi, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException
from minio import Minio
import pandas as pd
import json
import requests
import os
import glob

### 2. Run experiment and train model in MLFlow
For our example, we will use the elastic net wine example from [MLflow's tutorial](https://github.com/mlflow/mlflow/tree/master/examples/sklearn_elasticnet_wine).

Let's load the data to see what's inside:

In [4]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
# Wine Quality Sample
def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [7]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.793164022927685
  MAE: 0.6271946374319586
  R2: 0.10862644997792625


In [8]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7336400911821402
  MAE: 0.5643841279275428
  R2: 0.23739466063584158


In [15]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984665
  R2: 0.2799376066653344


Each of these models can actually be found on the `mlruns` folder:

In [9]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.8.0-1).
The following package was automatically installed and is no longer required:
  libfwupdplugin1
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [10]:
!tree -L 1 mlruns/0

mlruns/0
├── 280a5311590b4c5cb10b052cc6617819
├── 3061e588ff084adc8dc94d2193ea9b5b
├── 3638146d4e1449d3b133ad4d1484e357
├── 3fed2065421e403287d622d769e246ef
├── 4b3cb1e9f7c44223b2e64e67ba615958
├── 798e35d47d80437b9decf83583a41b21
├── 7b1acdcccf1c4fc28934865008a6319c
├── 8d944bf6f329461dba5e8fce94b3a02f
├── ab7b583a5ead4aec806d8c3c0d75fd55
├── afbc782902874e8c9edb137666a5f6a5
├── ce031de6fb36488b89c05063c5746827
├── d3136a7c576544c2bc136450ac0a51ac
└── meta.yaml

12 directories, 1 file


Inside each of these folders, MLflow stores the parameters we used to train our model, any metric we logged during training, and a snapshot of our model. If we look into one of them, we can see the following structure:

In [11]:
!tree mlruns/0/$(ls mlruns/0 | head -1)

mlruns/0/280a5311590b4c5cb10b052cc6617819
├── artifacts
│   └── model
│       ├── MLmodel
│       ├── conda.yaml
│       ├── environment.tar.gz
│       ├── model.pkl
│       ├── python_env.yaml
│       └── requirements.txt
├── meta.yaml
├── metrics
│   ├── mae
│   ├── r2
│   └── rmse
├── params
│   ├── alpha
│   └── l1_ratio
└── tags
    ├── mlflow.log-model.history
    ├── mlflow.source.git.commit
    ├── mlflow.source.name
    ├── mlflow.source.type
    └── mlflow.user

5 directories, 17 files


In particular, we are interested in the MLmodel file stored under artifacts/model:

In [12]:
!cat mlruns/0/$(ls mlruns/0 | head -1)/artifacts/model/MLmodel

artifact_path: model
flavors:
  python_function:
    env: conda.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    python_version: 3.7.4
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 0.20.3
mlflow_version: 1.28.0
model_uuid: 342c5d1852ff401eab8f492691f9c559
run_id: 280a5311590b4c5cb10b052cc6617819
utc_time_created: '2022-09-06 18:44:21.511028'


We can also view the dependencies that will be required to serve the model

In [13]:
!cat mlruns/0/$(ls mlruns/0 | head -1)/artifacts/model/conda.yaml

channels:
- conda-forge
dependencies:
- python=3.7.4
- pip<=22.1.2
- pip:
  - mlflow
  - cloudpickle==2.1.0
  - scikit-learn==0.20.3
name: mlflow-env


This file stores the details of how the model was stored. With this information (plus the other files in the folder), we are able to load the model back. Seldon’s MLflow server will use this information to serve this model.

### 4. Push artefacts to MinIO

In [14]:
RUN = "280a5311590b4c5cb10b052cc6617819"

In [15]:
minioClient = Minio("35.204.34.216:9000", "admin", "F]L?yy!]&J%_-27t", secure=False)

In [16]:
minioClient.list_buckets()

[Bucket('aaai'),
 Bucket('aav-ilse'),
 Bucket('deploy-audit'),
 Bucket('jorik'),
 Bucket('jorik2'),
 Bucket('jorikmlflowexample'),
 Bucket('models'),
 Bucket('nnlp-mlflow-test'),
 Bucket('nnlptestn'),
 Bucket('pkl'),
 Bucket('tayfun')]

In [54]:
def upload_local_directory_to_minio(local_path: str, bucket_name: str, folder_name: str):
    assert os.path.isdir(local_path)

    for local_file in glob.glob(local_path + '/**'):
        local_file = local_file.replace(os.sep, "/")
        if not os.path.isfile(local_file):
            upload_local_directory_to_minio(
                local_file, bucket_name)
        else:
            remote_path = os.path.join(
                local_file[1 + len(local_path):])
            remote_path = remote_path.replace(
                os.sep, "/")
            remote_path = folder_name + "/" + remote_path
            minioClient.fput_object(bucket_name, remote_path, local_file)

In [44]:
upload_local_directory_to_minio(f"mlruns/0/{RUN}/artifacts/model", "models", "default")

### 5. Deploy via the SDK

Create a function to authenticate against the cluster

In [17]:
SD_IP = "nn-group.cloud.seldon.io"

config = Configuration()
config.host = f"https://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"https://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = ">+HCA#AStf*NY$bD"
config.auth_method = "client_credentials" #change this value to "auth_code" to leverage user credentials

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

#### Method 1: Deploy with the Seldon protocol

During initialisation, the built-in reusable server will create the Conda environment specified on your conda.yaml file.  However, because this build can take some time, we must extend the initialDelaySeconds and failureThreshold for the probes on our deployment.

To use the built-in MLflow server the following pre-requisites need to be met:

* Your MLmodel artifact folder needs to be accessible remotely (e.g. in minio).

* Your model needs to be compatible with the python_function flavour.

* Your MLproject environment needs to be specified using Conda.

In [18]:
DEPLOYMENT_NAME = "wine-andrew-seldon"
NAMESPACE = "seldon-gitops"
MODEL_LOCATION = "minio:models/default"
SECRET_NAME = "minio-bucket"

mldeployment = {
  "apiVersion": "machinelearning.seldon.io/v1alpha2",
  "kind": "SeldonDeployment",
  "metadata": {
    "name": f"{DEPLOYMENT_NAME}",
    "namespace": NAMESPACE
  },
  "spec": {
    "name": f"{DEPLOYMENT_NAME}",
    "protocol":"seldon",
    "predictors": [
      {
        "componentSpecs": [
          {
            "spec": {
              "containers": [
                {
                  "name": f"{DEPLOYMENT_NAME}-container",
                  # We are setting high failureThreshold as installing conda dependencies
                  # can take a long time and we want to avoid k8s killing the container prematurely
                  "livenessProbe": {
                    "initialDelaySeconds": 100,
                    "failureThreshold": 300,
                    "periodSeconds": 5,
                    "successThreshold": 1,
                    "httpGet": {
                      "path": "/health/ping",
                      "port": "http",
                      "scheme": "HTTP"
                    }
                  },
                  "readinessProbe": {
                    "initialDelaySeconds": 100,
                    "failureThreshold": 300,
                    "periodSeconds": 5,
                    "successThreshold": 1,
                    "httpGet": {
                      "path": "/health/ping",
                      "port": "http",
                      "scheme": "HTTP"
                    }
                  }
                }
              ]
            }
          }
        ],
        "graph": {
          "children": [],
          "implementation": "MLFLOW_SERVER",
          "modelUri": f"{MODEL_LOCATION}",
          "envSecretRefName": f"{SECRET_NAME}",
          "name": f"{DEPLOYMENT_NAME}-container"
        },
        "name": "default",
        "replicas": 1
      }
    ]
  }
}

In [44]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

{'api_version': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'wine-andrew-seldon',
              'namespace': 'seldon-gitops',
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'annotations': None,
          'name': 'wine-andrew-seldon',
          'oauth_key': None,
          'oauth_secret': None,
          'predictors': [{'annotations': None,
                          'component_specs': [{'hpa_spec': None,
                                               'keda_spec': Non

You can now test a prediction in the Seldon Deploy UI by using the following payload:

```
{"data": {"names": [], "ndarray": [[7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8]]}}
```

You should see a 200 response

#### Method 2: Deploy with the v2 protocol
In order to use the v2 protocol, it is best to use conda pack to locally save the conda environment to a tar file.  The initialiser can then use this to install required dependencies into the container.

In [25]:
!conda pack -o mlruns/0/$(ls mlruns/0 | head -1)/artifacts/model/environment.tar.gz -f

Packing environment at '/home/andrew/.asdf/installs/python/miniconda3-4.7.12/envs/mlflow-wine' to 'mlruns/0/280a5311590b4c5cb10b052cc6617819/artifacts/model/environment.tar.gz'
[########################################] | 100% Completed |  2.9s


Push the environment.tar.gz file to MinIO

In [26]:
minioClient.fput_object("models", "default/environment.tar.gz", f"mlruns/0/{RUN}/artifacts/model/environment.tar.gz")

It should be enough to simply specify the v2 protocol.  You no longer need to adjust the liveness and readiness probes.  Note that this deployment can also be done via the UI in Seldon Deploy.

In [27]:
DEPLOYMENT_NAME = "wine-andrew-v2"
NAMESPACE = "seldon-gitops"
MODEL_LOCATION = "minio:models/default"
SECRET_NAME = "minio-bucket"


mldeployment = {
  "apiVersion": "machinelearning.seldon.io/v1alpha2",
  "kind": "SeldonDeployment",
  "metadata": {
    "name": f"{DEPLOYMENT_NAME}",
    "namespace": NAMESPACE
  },
  "spec": {
    "protocol": "v2",
    "name": f"{DEPLOYMENT_NAME}",
    "predictors": [
      {
        "graph": {
          "children": [],
          "implementation": "MLFLOW_SERVER",
          "modelUri": f"{MODEL_LOCATION}",
          "envSecretRefName": f"{SECRET_NAME}",
          "name": f"{DEPLOYMENT_NAME}-container"
        },
        "name": "default",
        "replicas": 1
      }
    ]
  }
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

### 6. Run an inference request

In [ ]:
inference_request = {
    "parameters": {
        "content_type": "pd"
    },
    "inputs": [
        {
          "name": "fixed acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [7.4],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "volatile acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.7000],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "citric acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "residual sugar",
          "shape": [1],
          "datatype": "FP32",
          "data": [1.9],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "chlorides",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.076],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "free sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [11],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "total sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [34],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "density",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.9978],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "pH",
          "shape": [1],
          "datatype": "FP32",
          "data": [3.51],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "sulphates",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.56],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "alcohol",
          "shape": [1],
          "datatype": "FP32",
          "data": [9.4],
          "parameters": {
              "content_type": "np"
          }
        },
    ]
}

In [ ]:
endpoint = ""
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))